# Making and breaking promises

**Learning outcomes**

<li> Explain how global information and promises are useful for algorithm design.

Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

Our little speedup in the previous node depended on a special fact about the function $f$ encoding the lock: there was only one secret combination $s$. But if we test a pair of combinations $|x0\rangle , |x1\rangle$ which both open the lock, we generate an undetectable phase once more:

Is our algorithm now useless? To open the lock, we just need a single solution. Our strategy of testing in pairs is still guaranteed to work (eventually), provided there are an odd number of solutions. Even if we miss the solutions that come in pairs, there will always be one left over! But on the other hand, if the number of solutions is even, we might get unlucky, with solutions pairing up into a bunch of unobservable phases.

![](https://codebook.xanadu.ai/pics/odd.svg)




There are two ways to view this algorithmic quirk. First, if we are told in advance that our lock has an odd number of solutions, we can test in pairs, confident we will eventually encounter a solution. But if we don't know the parity of the number (i.e., whether it is odd or even), we can test in pairs and discover it! Each time we observe a phase change, we know we have encountered a single solution, and otherwise, we have encountered either two or zero solutions, which are equal modulo $2$. So we simply test in pairs and count the number of phase changes we were able to observe. The parity of the solution set is the parity of this count. Once again, this is twice as fast as the classical algorithm, which must separately test each combination. This gives a circuit for determining the parity:


![](https://codebook.xanadu.ai/pics/flowchart2.svg)



Our pair-testing algorithm suggests that quantum computers might be good at determining global information about a function, like the parity of the solution set. Although we can't produce the observation we want from the exponentially many possibilities, we might be able to gather global information from all the different branches and combine them in an observable way. The pair-testing algorithm also tells us that, conversely, global assumptions about the function (e.g., that the number of solutions is odd) can lead to very different performance guarantees for our algorithms. These global assumptions about  are called promises, and play an important role in quantum algorithms.

![](https://codebook.xanadu.ai/pics/pair-test-circuit.svg)


### Codercise A.4.1 

Implement the circuit above, but now for `how_many` solutions instead of one. You will first need to implement the multi-solution version of the oracle matrix, `multisol_oracle_matrix`(combos), which takes a list of bit strings (representing different solutions) and returns the associated oracle in matrix form.

In [2]:

import pennylane as qml
import numpy as np

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

def multisol_oracle_matrix(combos):
    """Return the oracle matrix for a set of solutions.

    Args:
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    indices = [np.ravel_multi_index(combo, [2]*len(combo)) for combo in combos]
    ##################
    # YOUR CODE HERE #
    ##################
    my_array = np.identity(2**len(combos[0])) # Create the identity matrix
    for i in indices:
        my_array[i,i] = -1
    return my_array

@qml.qnode(dev)
def multisol_pair_circuit(x_tilde, combos):
    """Implements the circuit for testing a pair of combinations labelled by x_tilde.
    
    Args:
        x_tilde (list[int]): An (n_bits - 1)-bit string labelling the pair to test.
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    ##################
    # YOUR CODE HERE #
    ##################
    qml.Hadamard(wires=n_bits-1)
    qml.QubitUnitary(multisol_oracle_matrix(combos), wires = [i for i in range(n_bits)])
    qml.Hadamard(wires=n_bits-1)

    return qml.probs(wires=n_bits-1)


### Codercise A.4.2

 Use `multisol_pair_circuit(x_tilde, combos)` to create a routine for checking the parity of a lock.

In [4]:
def parity_checker(combos):
    """Use multisol_pair_circuit to determine the parity of a solution set.

    Args:
        combos (list[list[int]]): A list of secret combinations.

    Returns: 
        int: The parity of the solution set.
    """
    parity = 0
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]
    for x_tilde in x_tildes:

        ##################
        # YOUR CODE HERE #
        ##################

        # IMPLEMENT PARITY COUNTING ALGORITHM
        p0,p1 = multisol_pair_circuit(x_tilde, combos)
        if np.isclose(1,p1):
            parity += 1
    return parity % 2


We are beginning to see how global information, or promises, can be used to exploit superposition, and conversely, superpositions can be used to learn global information.